# This is simple NN example
- This example will require Keras setup to run.
- Keras=1.2
- Tensorflow: 1.14
- Python 3.7

In [1]:
# Some Imports
from sklearn import datasets
from mango import Tuner, scheduler

# Search Space

In [2]:
"""
Defining only the number of layers in the param_space.

During Neural Network training: hidden dimensions will be decided.
"""

# n_neighbors can vary between 1 and 50, with different choices of algorithm
param_space = dict(num_of_layers=range(1, 7))

# Load the Data

In [3]:
iris = datasets.load_iris()

# Take only the first two features.
X = iris.data
y = iris.target

In [4]:
print(X.shape, y.shape)

(150, 4) (150,)


# Convert y to one-hot encoded


In [5]:
from keras.utils import np_utils

y = np_utils.to_categorical(y, 3)

print(y.shape)

Using TensorFlow backend.


(150, 3)


# NN model using Keras

- Hidden dimensions are function of number of layers

In [6]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def get_model(num_of_layers):
    global X, y
    model = Sequential()
    model.add(Dense(4, input_dim=X.shape[1], activation='relu'))
    
    
    # Hidden dimension of layers = 4*num_of_layers
    for i in range(num_of_layers):
         model.add(Dense(4*num_of_layers, activation='relu'))
        
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', 
                      optimizer='adam', 
                      metrics=['accuracy'])
    
    return model


In [7]:
## Uncomment to see the model

# model = get_model(5)
# model.summary()

In [8]:
# Scheduler.
#We return the train accuracy after 10 epochs

@scheduler.serial
def objective(**params):
    global X, y
    model = get_model(**params)
    
    #train the clf.
    model.fit(X, y,
          batch_size=32,
          epochs=100,
          verbose=0)
    
    score = model.evaluate(X,y, verbose=0)
      
    return score[1]


In [9]:
conf_dict = dict(num_iteration=5)

tuner = Tuner(param_space, objective, conf_dict)
results = tuner.maximize()
print('best parameters:', results['best_params'])
print('best accuracy:', results['best_objective'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



best parameters: {'num_of_layers': 5}
best accuracy: 0.97333336


In [10]:
results

{'random_params': array([{'num_of_layers': 6}], dtype=object),
 'random_params_objective': array([0.93333334], dtype=float32),
 'params_tried': array([{'num_of_layers': 6}, {'num_of_layers': 1}, {'num_of_layers': 4},
        {'num_of_layers': 5}, {'num_of_layers': 3}, {'num_of_layers': 2}],
       dtype=object),
 'objective_values': array([0.93333334, 0.33333334, 0.9533333 , 0.97333336, 0.9533333 ,
        0.9533333 ], dtype=float32),
 'surrogate_values': array([0.93333334, 0.77340587, 0.72558583, 0.98862434, 0.8273985 ,
        0.7403795 ]),
 'best_objective': 0.97333336,
 'best_params': {'num_of_layers': 5}}